# Purpose
This code will: 
1. Retrieve the power consumption files (JSON format) from the Data Lake
1. Create the "iotSmartGrid.powerConsumption" table
1. Create a Dashboard to show the total power consumption and a map to show the power usage for each zipcode.

# Configuration
To use this sample code, you will have to provide the Data Lake storage account key in Cmd2. Ideally, you should create a secretScope to store the key value. 
For more information, please see this link - https://learn.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#create-a-databricks-backed-secret-scope

In [ ]:
# Configuring my account key so Databricks can access the Data lake.

spark.conf.set("fs.azure.account.key.sbdemostaiotsmartgrid.dfs.core.windows.net", 
               dbutils.secrets.get(scope="sbdemokviotsmartgrid", key="sbdemostaiotsmartgrid"))

In [ ]:
# Read the JSON Smart Grid power data from the Data Lake

gridDataDf = spark.read.json("abfss://landing@sbdemostaiotsmartgrid.dfs.core.windows.net/iotSmartGridData/*.json")

In [ ]:
# Display the data. This dataset has 4 columns: 
# 1. measurementDate is a timestamp.
# 2. measurementInKwh stores the power usage in KWh.
# 3. meterId stores the Meter ID that reports the usage.
# 4. zipCode stores the area this meter serves.

display(gridDataDf)

In [ ]:
%sql
-- Create iotSmartGrid if database/schema does not exist.

CREATE SCHEMA IF NOT EXISTS iotSmartGrid;

In [ ]:
# Create/Overwrite the powerConsumption table to store the data in gridDataDf
gridDataDf.write.mode("overwrite").format("delta").saveAsTable("iotSmartGrid.powerConsumption")

In [ ]:
%sql
SELECT sum(measurementInKWh)
FROM iotSmartGrid.powerConsumption;

In [ ]:
%sql
SELECT pc.zipCode, latlong.lat, latlong.long, sum(measurementInKWh) AS usageKWh
FROM iotSmartGrid.powerConsumption pc
JOIN iotsmartgrid.zip_code_long_lat latlong ON (pc.zipCode = latlong.zipCode)
GROUP BY pc.zipCode, latlong.lat, latlong.long;